#### Dataframe Joins and Optimizations
SparkSQL builds in several optimizations while performing joins and other operations on dataframes.

We can check out the execution plan of SparkSQL for any operation using the function "explain()"

When one of the dataframes in the join operation is small enough to fit into a workernode then Spark SQL plans to "broadcast" this dataframe. This is generally referred to as BroadcashJoin. When a dataframe is "broadcasted" then a copy of it is sent to all worker nodes. This avoids shuffling of data when joining it with the larger dataframe as the smaller dataframe is compltely available locally for each partition of the larger dataframe.

The above optimization is done automatically by SparkSQL's built-in optimizer. However we can also check to ensure it is done and in case it is not done it can be done manually also as shown in the example below.

The setting "spark.sql.autoBroadcastJoinThreshold" determines whether a dataframe is small enough to be broadcast or not. The default of this setting is 10 MB (10485760). We can override this when we know that the smaller dataframe is just above this threshold and can be managed to be fit into each worker node. 

In [0]:
# sc.defaultParallelism - returns default level of parallelism defined on SparkContext.By default it is number of cores available to application.
spark.sparkContext.defaultParallelism

Create DataFrame from data source - csv file

In [0]:
customerDF = spark.read.load("/FileStore/tables/customers.tsv", format="csv", sep="\t", inferSchema="true")
salestxnDF = spark.read.load("/FileStore/tables/salestxns.tsv", format="csv", sep="\t", inferSchema="true")

Different types of operations on DataFrames

In [0]:
customerDF.printSchema()

In [0]:
salestxnDF.printSchema()

In [0]:
new_names_cDF = ['customer_id','customer_name','customer_city','customer_state','customer_zipcode']

In [0]:
new_names_sDF = ['salestxn_id','category_id','category_name','product_id','product_name','product_price','product_quantity','customer_id']

In [0]:
cDF2=customerDF.toDF(*new_names_cDF)

In [0]:
sDF2=salestxnDF.toDF(*new_names_sDF)

In [0]:
cDF2.printSchema()

In [0]:
cDF2.show(5)

In [0]:
cDF2.count()

In [0]:
sDF2.printSchema()

In [0]:
sDF2.show(5)

In [0]:
cDF2.rdd.getNumPartitions()

In [0]:
sDF2.count()

In [0]:
sDF2.rdd.getNumPartitions()

Join the two dataframes on customer_id column

In [0]:
jDF1=cDF2.join(sDF2,cDF2.customer_id==sDF2.customer_id)

In [0]:
jDF1.printSchema()

In [0]:
jDF1.show(5)

Use explain() function to see the type of join used by Spark SQL

In [0]:
jDF1.explain()

Using explain(True) function gives the outline of the initial logical plans besides the final physical plan used for executing the join operation

In [0]:
jDF1.explain(True)

In [0]:
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

By setting this value to -1 broadcasting can be disabled.

In [0]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

In [0]:
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

In [0]:
jDF2=cDF2.join(sDF2,cDF2.customer_id==sDF2.customer_id)

In [0]:
# Since autoBroadcastJoinThreshold is disabled by setting it to -1, physical plan shows BroadCastHashJoin it is not used.
jDF2.explain()

We can notice that broadcast join is not used now as the threshold is disabled.

In [0]:
jDF2.printSchema()

In [0]:
jDF2.show(5)

In [0]:
from pyspark.sql import functions as F

The lines below show how to mark a dataframe for broadcast in case required to be done manually.

In [0]:
jDF3=cDF2.hint("broadcast").join(sDF2,cDF2.customer_id==sDF2.customer_id)

In [0]:
jDF3.explain()

In [0]:
jDF3.printSchema()

In [0]:
jDF3.show(5)

In [0]:
jDF4=F.broadcast(cDF2).join(sDF2,cDF2.customer_id==sDF2.customer_id)

In [0]:
jDF4.explain()

In [0]:
jDF4.printSchema()

In [0]:
jDF4.show(5)

In [0]:
cDF2.cache()

In [0]:
cDF2.count()

In [0]:
sDF2.cache()

In [0]:
sDF2.count()

In [0]:
jDF5=F.broadcast(cDF2).join(sDF2,cDF2.customer_id==sDF2.customer_id)

In [0]:
jDF5.count()

In [0]:
# In the physical plan below we can see "InMemoryTableScan" being used for both the dataframes
# which is not the case in the previous one.
# This means that the data is read from the cache() insted of re-building the dataframes from scratch.

jDF5.explain()

In [0]:
jDF5.show(5)

In [0]:
# In the statement below, after join we are doing a filter and also we are doing select of only 5 columns.
# In the physical plan shown after the explain statement, we can see filter is done ahead of the join.
# This means that the query execution engine executes projection and then filter and only after that join is executed,
# which is more optimal way of executing the whole statement.

jDF6=cDF2.hint("broadcast").join(sDF2,cDF2.customer_id==sDF2.customer_id).\
select('customer_name','customer_city','product_name','product_price','product_quantity').where(cDF2.customer_state=='IL')

In [0]:
jDF6.show(5)

In [0]:
jDF6.explain()

In [0]:
sDF2.rdd.getNumPartitions()

In [0]:
sDF3=sDF2.repartition(16,'customer_id')

In [0]:
sDF3.rdd.getNumPartitions()

In [0]:
sDF3.cache()

In [0]:
sDF3.show(5)

In [0]:
sDF3.show(5)

In [0]:
cDF2.rdd.getNumPartitions()

In [0]:
cDF3=cDF2.repartition(1,'customer_id')

In [0]:
cDF3.rdd.getNumPartitions()

In [0]:
cDF3.cache()

In [0]:
cDF3.show(5)

In [0]:
cDF3.show(5)

In [0]:
jDF7=cDF3.hint("broadcast").join(sDF3,cDF3.customer_id==sDF3.customer_id).\
select('customer_name','customer_city','product_name','product_price','product_quantity').where(cDF3.customer_state=='IL')

In [0]:
jDF7.show(5)

In [0]:
jDF7.explain()